In [ ]:
import numpy as np 
import pandas as pd
import os
import shutil
from functions.space import Space
from functions.utils import LoadBar

In [ ]:
# Define Variables
coordinates  = np.array(pd.read_csv('Dimensionality_reduction\Adjency_matrix\\coordinates.csv', header = None))
point_id = np.argsort(coordinates[:, -1].astype(int))
connectivity = pd.read_csv('Dimensionality_reduction\Adjency_matrix\\adjacency_matrix.csv')
edge_indices = connectivity[['point_i', 'point_j']]
edge_distances  = np.load("Dimensionality_reduction\Adjency_matrix\\edge_weight_array.npy") # connectivity[['distance']]
output_dir_path = 'output\\'

In [ ]:
DELETE_ALL = True  # Flag to determine whether to delete all previous outputs

if DELETE_ALL:
    try:
        shutil.rmtree(output_dir_path)  # Remove the output directory and all its contents
        os.makedirs(output_dir_path)    # Recreate the output directory
    except Exception as e:
        print(f"Error removing folder {output_dir_path}")  # Print error if removal fails
        raise e

np.random.seed(0)  # Set random seed for reproducibility
dataset_pressure_gradient = np.load('input\\agard_doe_r002_dataset_pressure_gradient.npy')  # Load pressure gradient dataset
# Compute gradient magnitude
gradient_magnitude = np.linalg.norm(dataset_pressure_gradient, axis=-1)  # Calculate the magnitude of the gradient for each sample

compression_ratio = 4  # Set the compression ratio

npoints1 = int(gradient_magnitude.shape[-1]/compression_ratio)  # Number of points after first compression
npoints2 = int(npoints1/compression_ratio)  # Number of points after second compression

loadbar = LoadBar()  # Initialize the loading bar

for sample in range(gradient_magnitude.shape[0]):  # Iterate over all samples
    output_dir = output_dir_path + f'Sample_{sample:03d}\\'  # Define output directory for the current sample
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)  # Create output directory if it doesn't exist
    else: continue  # Skip if directory already exists
    loadbar.tick()  # Update the loading bar
    ed_dis = pd.DataFrame({"distance":edge_distances[sample,:]})  # Create DataFrame for edge distances

    space1 = Space()  # Initialize first Space object
    print(loadbar.loadBarI(sample,gradient_magnitude.shape[0],suffix="Creating space1...", prefix=f'Sample_{sample:03d}:'), end = "\r")
    space1.create(npoints1, gradient_magnitude[sample,:], coordinates, point_id, edge_indices, ed_dis)  # Create first reduced space

    print(loadbar.loadBarI(sample,gradient_magnitude.shape[0],suffix="Saving space1...", prefix=f'Sample_{sample:03d}:'), end = "\r")
    space1.save(output_dir+"space1")  # Save first reduced space

    space2 = Space()  # Initialize second Space object
    print(loadbar.loadBarI(sample,gradient_magnitude.shape[0],suffix="Creating space2...", prefix=f'Sample_{sample:03d}:'), end = "\r")
    space2.create(npoints2, space1.magnitude_reduced, space1.coordinates_reduced, space1.point_id_reduced, space1.edge_indices_reduced, space1.edge_distances_reduced, space2=True)  # Create second reduced space

    print(loadbar.loadBarI(sample,gradient_magnitude.shape[0],suffix="Saving space2...", prefix=f'Sample_{sample:03d}:'), end = "\r")
    space2.save(output_dir+"space2")  # Save second reduced space

    loadbar.tock()  # Finalize the loading bar for this iteration
